In [1]:
!pip install pyspark

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [4]:
spark = SparkSession.builder.getOrCreate()

In [5]:
produtos = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/produtos.csv', header=True, inferSchema=True)
vendedores = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/vendedores.csv', header=True, inferSchema=True)
clientes = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/clientes.csv', header=True, inferSchema=True)
itens_pedido = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/itens_pedido.csv', header=True, inferSchema=True)
pagamentos_pedido = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/pagamentos_pedido.csv', header=True, inferSchema=True)
avaliacoes_pedido = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/avaliacoes_pedido.csv', header=True, inferSchema=True)
pedidos = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/pedidos.csv', header=True, inferSchema=True)

In [6]:
# Obtendo uma visão geral das tabelas
produtos.show(2)
vendedores.show(2)
clientes.show(2)
itens_pedido.show(2)
pagamentos_pedido.show(2)
avaliacoes_pedido.show(2)
pedidos.show(2)

+--------------------+-----------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|          id_produto|categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|
+--------------------+-----------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|1e9e8ef04dbcff454...|       perfumaria|                  40|                      287|                       1|           225|                    16|               10|                14|
|3aa071139cb16b67c...|            artes|                  44|                      276|                       1|          1000|                    30|               18|                20|
+--------------------+-----------------+--------------------

In [7]:
# Criando tabelas temporárias para as manipulações sem alterar as tabelas originais
produtos.createOrReplaceTempView('produtos_temp')
vendedores.createOrReplaceTempView('vendedores_temp')
clientes.createOrReplaceTempView('clientes_temp')
itens_pedido.createOrReplaceTempView('itens_pedido_temp')
pagamentos_pedido.createOrReplaceTempView('pagamentos_pedido_temp')
avaliacoes_pedido.createOrReplaceTempView('avaliacoes_pedido_temp')
pedidos.createOrReplaceTempView('pedidos_temp')

In [10]:
spark.catalog.listDatabases()

[Database(name='default', catalog='spark_catalog', description='default database', locationUri='file:/content/spark-warehouse')]

In [11]:
spark.catalog.listTables()

[Table(name='avaliacoes_pedido_temp', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='clientes_temp', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='itens_pedido_temp', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='pagamentos_pedido_temp', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='pedidos_temp', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='produtos_temp', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='vendedores_temp', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [12]:
# Criando um DataFrame a partir de uma query sql

itens_pedido_produto_df = spark.sql("""
    SELECT pr.id_produto, pr.categoria_produto, ip.valor_frete, ip.preco
    FROM itens_pedido_temp ip
    JOIN produtos_temp pr ON ip.id_produto = pr.id_produto
""")

itens_pedido_produto_df.printSchema()
itens_pedido_produto_df.show(5)

root
 |-- id_produto: string (nullable = true)
 |-- categoria_produto: string (nullable = true)
 |-- valor_frete: double (nullable = true)
 |-- preco: double (nullable = true)

+--------------------+------------------+-----------+-----+
|          id_produto| categoria_produto|valor_frete|preco|
+--------------------+------------------+-----------+-----+
|4244733e06e7ecb49...|        cool_stuff|      13.29| 58.9|
|e5f2d52b802189ee6...|          pet_shop|      19.93|239.9|
|c777355d18b72b67a...|  moveis_decoracao|      17.87|199.0|
|7634da152a4610f15...|        perfumaria|      12.79|12.99|
|ac6c3623068f30de0...|ferramentas_jardim|      18.14|199.9|
+--------------------+------------------+-----------+-----+
only showing top 5 rows



In [13]:
# Também é possível passar a query para o PySpark como uma variável:

query = """
    SELECT ip.id_produto, AVG(ap.nota_avaliacao) AS media_avaliacao
    FROM avaliacoes_pedido_temp ap
    JOIN itens_pedido_temp ip ON ap.id_pedido = ip.id_pedido
    GROUP BY ip.id_produto
"""

avaliacao_produto_df = spark.sql(query)
avaliacao_produto_df.printSchema()
avaliacao_produto_df.show(5)

root
 |-- id_produto: string (nullable = true)
 |-- media_avaliacao: double (nullable = true)

+--------------------+------------------+
|          id_produto|   media_avaliacao|
+--------------------+------------------+
|0b0172eb0fd18479d...|             3.875|
|42a2bd596fda1baef...|               4.0|
|460a66fcc404a3d73...|3.8333333333333335|
|30360c8b0b2ac6918...| 4.666666666666667|
|13b4ff901d43edec6...|               5.0|
+--------------------+------------------+
only showing top 5 rows



In [14]:
# é possível realizar o Join normalmente entre os novos DataFrames

detalhe_produto_df = avaliacao_produto_df.join(itens_pedido_produto_df, 'id_produto')
detalhe_produto_df.show()

# isso é partucularmente útil pois é possível 'dividir a complexidade' entre o PySpark e a linguagem SQL conforme for mais conveniente

+--------------------+---------------+-----------------+-----------+-----+
|          id_produto|media_avaliacao|categoria_produto|valor_frete|preco|
+--------------------+---------------+-----------------+-----------+-----+
|0b0172eb0fd18479d...|          3.875|      eletronicos|      17.63|24.89|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      15.79|24.89|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      22.67| 28.9|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      16.92| 28.9|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      26.89| 28.9|
|0b0172eb0fd18479d...|          3.875|      eletronicos|       8.11|24.89|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      15.79|24.89|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      15.79|24.89|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      16.11| 28.9|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      12.48|24.89|
|0b0172eb0fd18479d...|   

In [15]:
# Salvar em diferentes formatos

# Salvar no Google Drive indicando o formato (ex. parquet)
detalhe_produto_df.write.mode('overwrite').option('header', 'true').parquet('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/output/detalhe_produto_parquet')

# Salvar como tabela localmente no warehouse do Spark (file:/content/spark-warehouse)
detalhe_produto_df.write.mode('overwrite').option('header', 'true').saveAsTable('detalhe_produto')

# Salvar como tabela no Google Drive (Cuidado com o Path!)
# Igual o anterior, mas precisa passar o parâmetro Path no .option
detalhe_produto_df.write.mode('overwrite').option('path', '../drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/output/table_produto').saveAsTable('detalhe_produto_table')

In [16]:
# lendo a tabela que foi salva no Warehouse
query = 'SELECT * FROM detalhe_produto'
teste_detalhe_produto = spark.sql(query)
teste_detalhe_produto.show()

+--------------------+---------------+-----------------+-----------+-----+
|          id_produto|media_avaliacao|categoria_produto|valor_frete|preco|
+--------------------+---------------+-----------------+-----------+-----+
|0b0172eb0fd18479d...|          3.875|      eletronicos|      17.63|24.89|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      15.79|24.89|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      22.67| 28.9|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      16.92| 28.9|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      26.89| 28.9|
|0b0172eb0fd18479d...|          3.875|      eletronicos|       8.11|24.89|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      15.79|24.89|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      15.79|24.89|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      16.11| 28.9|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      12.48|24.89|
|0b0172eb0fd18479d...|   

In [17]:
# Lendo a tabela que foi salva no Google Drive
detalhe_produto_table_df = spark.read.option('path', '../drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/output/table_produto').table('detalhe_produto_table')
detalhe_produto_table_df.show()

+--------------------+---------------+-----------------+-----------+-----+
|          id_produto|media_avaliacao|categoria_produto|valor_frete|preco|
+--------------------+---------------+-----------------+-----------+-----+
|0b0172eb0fd18479d...|          3.875|      eletronicos|      17.63|24.89|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      15.79|24.89|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      22.67| 28.9|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      16.92| 28.9|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      26.89| 28.9|
|0b0172eb0fd18479d...|          3.875|      eletronicos|       8.11|24.89|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      15.79|24.89|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      15.79|24.89|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      16.11| 28.9|
|0b0172eb0fd18479d...|          3.875|      eletronicos|      12.48|24.89|
|0b0172eb0fd18479d...|   

In [18]:
# Fazendo uma consulta direta apenas para visualização

spark.sql("""
    SELECT
        YEAR(data_compra_pedido) AS compra_anual,
        MONTH(data_compra_pedido) AS compra_mensal,
        COUNT(*) AS count
    FROM
        pedidos_temp
    GROUP BY
        compra_anual,
        compra_mensal
    ORDER BY
        compra_anual DESC,
        compra_mensal

""").show()

+------------+-------------+-----+
|compra_anual|compra_mensal|count|
+------------+-------------+-----+
|        2018|            1| 7269|
|        2018|            2| 6728|
|        2018|            3| 7211|
|        2018|            4| 6939|
|        2018|            5| 6873|
|        2018|            6| 6167|
|        2018|            7| 6292|
|        2018|            8| 6512|
|        2018|            9|   16|
|        2018|           10|    4|
|        2017|            1|  800|
|        2017|            2| 1780|
|        2017|            3| 2682|
|        2017|            4| 2404|
|        2017|            5| 3700|
|        2017|            6| 3245|
|        2017|            7| 4026|
|        2017|            8| 4331|
|        2017|            9| 4285|
|        2017|           10| 4631|
+------------+-------------+-----+
only showing top 20 rows



In [19]:
spark.stop()